In [1]:
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.metrics import confusion_matrix

## Defining the bucket 
s3 = boto3.resource('s3')
bucket_name = 'data-448'
bucket = s3.Bucket(bucket_name)

## Defining the csv file 
file_key = 'In_Class_Assignments/Iris.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
iris = pd.read_csv(file_content_stream)
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [2]:
## Creating Species_numb
iris['Species_numb'] = np.where(iris['Species'] == 'Iris-virginica', 1, np.where(iris['Species'] == 'Iris-versicolor', 2, 3))
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,Species_numb
0,1,5.1,3.5,1.4,0.2,Iris-setosa,3
1,2,4.9,3.0,1.4,0.2,Iris-setosa,3
2,3,4.7,3.2,1.3,0.2,Iris-setosa,3
3,4,4.6,3.1,1.5,0.2,Iris-setosa,3
4,5,5.0,3.6,1.4,0.2,Iris-setosa,3


In [3]:
## Defining the input & target variables
X = iris[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
Y = iris['Species_numb']

## Splitting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

## Standardize the input variables (0-1 scale)   
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [4]:
## one-vs-one strategy with logistic
one_vs_one_logit = OneVsOneClassifier(estimator = LogisticRegression()).fit(X_train, Y_train)

## Predicting on the test data-frame 
one_vs_one_logit_pred = one_vs_one_logit.predict(X_test)

## Comparing predictions with actuals
confusion_matrix(Y_test, one_vs_one_logit_pred)

array([[10,  0,  0],
       [ 1,  9,  0],
       [ 0,  0, 10]])

In [5]:
## one-vs-one strategy with decisiontree
one_vs_one_tree = OneVsOneClassifier(estimator = DecisionTreeClassifier(max_depth = 3)).fit(X_train, Y_train)

## Predicting on the test data-frame 
one_vs_one_tree_pred = one_vs_one_tree.predict(X_test)

## Comparing predictions with actuals
confusion_matrix(Y_test, one_vs_one_tree_pred)

array([[10,  0,  0],
       [ 1,  9,  0],
       [ 0,  0, 10]])

In [6]:
## From the above results, we see that both models more or less have the same 
## performance